In [ ]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB2 #260x260
import tensorflow.keras.layers as layers
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import Sequence
from tensorflow.keras import regularizers
import imgaug.augmenters as iaa
import imgaug

from sklearn.model_selection import train_test_split

In [ ]:
pd.set_option('display.max_colwidth', None)
#data_dir = '/run/user/1000/gvfs/smb-share:server=titan.local,share=datasets/kitchenware-classification'
data_dir = 'data'
seed = 42

In [ ]:
df_train_full = pd.read_csv(data_dir+'/train.csv', dtype={'Id': str})
df_extra = pd.read_csv(data_dir+'/data.csv', dtype={'Id': str})
df_forks = pd.read_csv(data_dir+'/forks_aug.csv', dtype={'Id': str})
#df_spoons = pd.read_csv(data_dir+'/spoon_aug.csv', dtype={'Id': str})
#df_plates = pd.read_csv(data_dir+'/plate_aug.csv', dtype={'Id': str})
#df_cups = pd.read_csv(data_dir+'/cup_aug.csv', dtype={'Id': str})
df_glasses = pd.read_csv(data_dir+'/glasses_aug.csv', dtype={'Id': str})
df_knife = pd.read_csv(data_dir+'/knife_aug.csv', dtype={'Id': str})
#df_train_full = pd.concat([df_train_full, df_extra, df_forks, df_spoons, df_plates, df_cups, df_glasses], ignore_index = True)
df_train_full = pd.concat([df_train_full, df_extra, df_forks, df_glasses, df_knife], ignore_index = True)
df_train_full['filename'] = data_dir+'/images/' + df_train_full['Id'] + '.jpg'
df_train_full = df_train_full.sample(frac = 1, random_state = seed)
df_train_full.reset_index(drop = True, inplace = True)
df_train_full.head()

In [ ]:
df_train_full.label.value_counts()

In [ ]:
#df_forks

In [ ]:
image_width = 260
image_height = 260
NUM_CLASSES = 6
batch_size = 32

In [ ]:
df_train, df_val, y_train, y_val = train_test_split(df_train_full, df_train_full['label'], test_size = 0.2, shuffle = True, stratify = df_train_full['label'], random_state = seed)
df_train

In [ ]:
histories = []
def combine_histories():
    history = {
        "loss": [],
        "val_loss": [],
        "accuracy": [],
        "val_accuracy": []
    }
    for h in histories:
        #print(history.keys())
        for k in history.keys():
            history[k] += h[k]
            #print(h[k])
    return history

In [ ]:
def plot_hist(hist):
    #print(hist)
    plt.plot(hist["accuracy"])
    plt.plot(hist["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()
    


In [ ]:
def compile_model(model, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    
    return model

In [ ]:
def build_model(num_classes, learning_rate):
    
    inputs = layers.Input(shape = (image_width, image_height, 3))
    x=inputs
    model = EfficientNetB2(include_top=False, input_tensor=x, weights="imagenet")
    #model = EfficientNetB2(include_top=False, input_tensor=x)
    #path_to_weights = "noisy_weights/adv.prop.notop-b2.h5"
    #model.load_weights(path_to_weights, by_name=True)

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.7
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred", kernel_regularizer=regularizers.l2(0.05))(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    
    return compile_model(model, learning_rate)

In [ ]:
def unfreeze_model(model, freeze, learning_rate):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = freeze

    #optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    #model.compile(
    #    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    #)
    return compile_model(model, learning_rate)

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    rescale=1/255.
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    rescale=1/255.,
)

In [ ]:
n_epochs = 5
learning_rate=5e-4
model = build_model(num_classes=NUM_CLASSES, learning_rate = learning_rate)

hist = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=val_generator
)

histories.append(hist.history)
history = combine_histories()
plot_hist(history)

In [ ]:
unfreeze_model(model, True, 1e-4)
n_epochs = 6
hist = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=val_generator
)

histories.append(hist.history)
history = combine_histories()
plot_hist(history)

In [ ]:
unfreeze_model(model, False, 2e-7)
n_epochs = 3
hist = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=val_generator
)

histories.append(hist.history)
history = combine_histories()
plot_hist(history)

In [ ]:
history.history

In [ ]:
df_test = pd.read_csv(data_dir+'/test.csv', dtype={'Id': str})
df_test['filename'] = data_dir+'/images/' + df_test['Id'] + '.jpg'
df_test.head()

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    target_size=(image_width, image_height),
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
y_pred = model.predict(test_generator)

In [ ]:
classes = np.array(list(train_generator.class_indices.keys()))
classes

In [ ]:
predictions = classes[y_pred.argmax(axis=1)]

In [ ]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len(data_dir+'/images/'):-4]
del df_submission['filename']

In [ ]:
df_submission[['Id', 'label']].to_csv('submissions/effb2_imnet_v8_more_forks.csv', index=False)

In [ ]:
!kaggle competitions submit kitchenware-classification -f submissions/effb2_imnet_v8_more_forks.csv -m 'validation: 0.9652'